# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

misschien handige code om te gebruiken:

%pwd (de data moet in je workdirectery staan)


In [181]:
# importeren libs
import numpy as np
import pandas as pd
import datetime
from operator import attrgetter

er moet ook nog een keer worden uitgelegd wat er in welk df staat en wat er met ieder df is gedaan voordat het geimporteerd is. !TODO

In [2]:
# importeren data
dfProjectTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Project tasks')
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Additional tasks')
dfCrew = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Crew')
dfRooms = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Rooms')
dfHiring = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Hiring')

Het schoonmaken van de cel, hierdoor kunnen we waardes makkelijker worden teruggevonden dan NaN

In [3]:
# aanpassen van de data (NaN -> 0 en x -> 1)
dfProjectTasks = dfProjectTasks.fillna(0)
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfCrew = dfCrew.fillna(0)
dfRooms = dfRooms.fillna(1)

dfProjectTasks.replace("x", 1, inplace=True)
dfAdditionalTasks.replace("x", 1, inplace=True)
dfCrew.replace("x", 1, inplace=True)


**Belangrijke info voor het plannen**
- De ingehuurde mensen hebben skill 1 en 8 
- kort inhuren is maximaal 60 uur
- het bedrijf is van 9-18 open
- een crewlid kan niet meer dan 3 uur achter elkaar werken (hoe lang is de pauze die moet worden genomen? >> half uur)
- een crewlid werkt nooit meer dan 7 uur op een dag
- kan een 'acquire skill' maar 1 keer per sprint worden gedaan of kunnen meerdere personen er een volgen? zoveel als je wilt 
- crewleden moet te samen aan de skills komen voor de opdracht.
- moeten de mensen die skill 1 nog niet hebben deze ook leren? hoeven niet te worden geleerd
- kan een crewlid eerst een uur aan het ene project werken, dit niet afronden en dan nog aan een ander project? >> bij project tasks moet het aan eengesloten zijn, bij additional tasks hoeft dit niet
- moet iedereen tegelijk aan het project werken >> moet!

Stel we maken een soort kalender, startend op 1 januari 2020. 
Hiervan maken we een dataframe met de volgende kolommen:
- datum
- de taak
- persoon die het uitvoerd
- begintijd 
- eindtijd
- Tijdsduur?
- Of het crewlid is ingehuurd (T/F)
- Kosten zodra het crewlid is ingehuurd
- Of het een werkdag is (T/F)

In [4]:
# het maken van een dataframe als kalender
data = pd.date_range("2020-01-01", periods=4*10*7, freq='D')

columns = ['Datum', 'Task','Crewlid', 'begintijd', 'eindtijd', 'tijdsduur', 'ingehuurd', 'kosten', 'werkdag']

dfKalender = pd.DataFrame(columns=columns)
dfKalender['Datum'] = data
dfKalender = dfKalender.fillna(0)
dfKalender.replace("0", False, inplace=True)


Het toevoegen of een dag een werkdag is. Er wordt in het weekend niet gewerkt dus deze dagen zullen afvallen.

In [5]:
dfKalender['werkdag'] = dfKalender['Datum'].apply(lambda Datum: Datum.isoweekday() in range (1,6))

Er moet een functie zijn die kan worden aangeroepen om te kijken of een taak door een bepaald persoon kan worden uitgevoerd.

Hier moeten de eisen van de opdracht worden meegegeven en een persoon. Vervolgens krijg je een boolean terug of het kan.

In [6]:
# een paar standaard variabele om de code leesbaar te maken.
rangeSkills = range(1,14)
rangeWorkdays = range(14,19)
rangeRooms = range(14,20)

functie waarbij de lijst van ReqSkill moet worden ingevuld van de Task en de skills van een persoon. De functie geeft een boolean terug of deze taak door deze persoon kan worden gedaan.

voorbeeld: <br>
OpdrachtUitvoeren(dfProjectTasks.iloc[0,rangeSkills],dfCrew.iloc[0,rangeSkills]) 
<br>uitkomst: False

In [7]:
def OpdrachtUitvoerenPersoonBool(eisen, persoon):
    for i in rangeSkills:
        result = (int(eisen[i-1]) <= 
                  int(persoon[i-1]))
        if result == False:
            break
    return result

de functie hieronder voegt de skills van personen samen en geeft de gezamelijke skills terug.

voorbeeld: PersonenSamenvoegen(dfCrew.iloc[0,rangeSkills],dfCrew.iloc[2,rangeSkills])
<br>
uitkomst: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]

In [8]:
def PersonenSamenvoegen(persoon1, persoon2 = [0]*13, persoon3 = [0]*13, persoon4 = [0]*13, persoon5 = [0]*13, persoon6 = [0]*13):
    combinedSkill = []
    surplus = 0 
    for i in rangeSkills:
        combinedSkill.append(int(1) if persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] > 0 else int(0))
        surplus = surplus + (persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] -1 
                         if persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] > 0 else int(0))
    combinedSkill.append(surplus)
    return combinedSkill

In [9]:
# het selecteren van alleen de tasks van sprint 1 en het dataframe maken met de columnnames en het resetten van de index
dfProjectTasks = dfProjectTasks[dfProjectTasks['Task'] != 0]
dfSprint1 = pd.DataFrame(dfProjectTasks[dfProjectTasks['Task'].str.contains('S1')], 
                         columns= list(dfProjectTasks)).reset_index(drop=True)


In [10]:
# het maken van de kolom crew en het aanpassen van het typen naar object zodat er een list of array kan worden toegevoegd aan die kolom.
dfSprint1.loc[:,'Crew']= [20]*len(dfSprint1)
dfSprint1['Crew'] = dfSprint1['Crew'].astype(object)

In [11]:
# het verzamelen van data van alle gezamelijke skills in staan tot aan 7 personen

dfCrewCombined = []
for i in range(0, len(dfCrew)-1): 
    t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills])
    t.append(i)
    t.append(20)
    t.append(20)
    t.append(20)
    t.append(20)
    t.append(20)
        
    for j in range(0, len(dfCrew)-1): # combinaties met twee mensen
        if i==j:
            break
        t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills])
        t.append(i)
        t.append(j)
        t.append(20)
        t.append(20)
        t.append(20)
        t.append(20)
        
        dfCrewCombined.append(t)

        for k in range(0, len(dfCrew)-1):
            if i==k or k==j:
                break
        
            t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills], dfCrew.iloc[k,rangeSkills])
            t.append(i)
            t.append(j)
            t.append(k)
            t.append(20)
            t.append(20)
            t.append(20)
            
            dfCrewCombined.append(t)
            
            for l in range(0, len(dfCrew)-1):
                if l == k or l == i or l == j :
                    break
                t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
                                        dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills])
                t.append(i)
                t.append(j)
                t.append(k)
                t.append(l)
                t.append(20)
                t.append(20)
                dfCrewCombined.append(t)

                for m in range(0, len(dfCrew)-1):
                    if m==l or m==k or m==j or m==i:
                        break
                    t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
                                            dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
                                            dfCrew.iloc[m,rangeSkills])
                    t.append(i)
                    t.append(j)
                    t.append(k)
                    t.append(l)
                    t.append(m)
                    t.append(20)
                    dfCrewCombined.append(t)
                    
                    for n in range(0, len(dfCrew)-1):
                        if n==m or n==l or n==k or n==j or n==i:
                            break
                        t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
                                                dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
                                                dfCrew.iloc[m,rangeSkills], dfCrew.iloc[n,rangeSkills])
                        t.append(i)
                        t.append(j)
                        t.append(k)
                        t.append(l)
                        t.append(m)
                        t.append(n)
                        dfCrewCombined.append(t)


                    
       

het dataframe dfCrewCombined is een dataframe met alle skills van de medewerkers gezamelijk. Zodra er bij een van de personen '20' staat, betekent dit dat er geen persoon voor die plek is toegevoegd. in het dataframe staan combinaties met twee personen tot zeven ('vo) personen.


In [12]:
# het maken van het dataframe dfCrewCombined
ar = np.array(dfCrewCombined, dtype=np.int16) 
dfCrewCombined = pd.DataFrame(ar, columns= ['Skill1', 'Skill2', 'Skill3', 'Skill4', 'Skill5', 'Skill6', 'Skill7', 'Skill8', 
                                'Skill9', 'Skill10', 'Skill11', 'Skill12', 'Skill13', 'Surplus', 'persoon1', 'persoon2', 
                                'persoon3', 'persoon4', 'persoon5', 'persoon6'])

In [13]:
# creeren van de kolom 'sum' zodat we weten hoeveel skill de crew gecombineerd heeft
dfCrewCombined['sum'] = dfCrewCombined.iloc[:,0:13].sum(axis = 1) 

In [14]:
# sorteren op de kolommen sum en surplus (overschot aan skills) zodat er zo min mogelijk overbodige skills worden gebruikt 
dfCrewCombined = dfCrewCombined.sort_values(['sum', 'Surplus'], ascending= [True, True])
dfCrewCombined = dfCrewCombined.reset_index(drop=True)

In [227]:
class ProjectUren:
    def __init__(self, project, uren):
        self.project = project
        self.uren = uren
        
class RoosterDag:
    
    def __init__(self, crew, datum, urenTeGaan=7, projectUren=['x', 'x']):
        self.crew = crew
        self.datum = datum
        self.urenTeGaan = urenTeGaan
        self.projectUren = ProjectUren(*projectUren)
         
        # stel we zetten in deze classe nog de velden 'nog te werken uren', 'projecten, aantal uur(in array)'
        # hierin zou een methode moeten met een max van 7 uur per dag >> wss met een setter
        
        

In [283]:
def Lijstmogelijkheden(crew, duration):
    lijstmogelijkheden = []
    for dag in werkdagen:
        if crew == dag.crew:
            if dag.urenTeGaan >= duration : # vier moet worden vervangen door task.duration
                lijstmogelijkheden.append(dag)
    if len(lijstmogelijkheden) == 0:
        lijstmogelijkheden.append(RoosterDag("geen medewerker",datetime.date(2100,1,1)))
                                  
    return lijstmogelijkheden

In [44]:
# # code voor het invullen van crew bij een task waarbij nog niet wordt gelet op of mensen al bezet zijn.
# werkdagen = []
# for j in range(0, len(dfSprint1)):
#     inplannen = False
#     i = 0
    
#     while inplannen == False:
#         if OpdrachtUitvoerenPersoonBool(dfSprint1.iloc[j,rangeSkills], dfCrewCombined.iloc[i,0:13]):
#             inplannen = True
#             dfSprint1.at[j,'Crew'] = np.array(dfCrewCombined.iloc[i,14:20])
            
#             for k in dfCrewCombined.iloc[i,14:20].tolist():
#                 if k ==20:
#                     break
                
#                 werkdagen.append(RoosterDag(k,"1-1-1990", 7- dfSprint1.Duration[j] , [dfSprint1.Task[j], dfSprint1.Duration[j]]))
#             i=0
#         else:
#             i = i + 1


In [236]:
def add_project(*projectUren): # de huidige moet worden mee gegeven en degene die je wilt toevoegen.
    # Hoeveel je er wilt toevoegen doet er niet toe.
    lijst = []
    for project in projectUren:
        if type(project) == list:
            lijst.append(ProjectUren(project[0], project[1]))
        else:
            if project.project != "x":
                lijst.append(project)
    return lijst

In [191]:
# vind een Roosterdag met crewlid en datum (dit zou uniek moeten zijn)
def VindRoosterdag(crew, datum):
    for dag in werkdagen:
        if crew == dag.crew:
            if dag.datum == datum: 
                return dag

Voorbeeld voor de functie add_project <br>
t = test2(["S",2]) <br>
t.projectUren = add_project(t.projectUren, ["D", 2]) <br>
<br><br>
for project in t.projectUren: <br>
    print (project.project) <br>
    
Uitkomst: <br>
S<br>
D<br>

In [286]:
# de eerste dag aanmaken voor iedereen
werkdagen = []
for i in range(0, len(dfCrew)):
    werkdagen.append(RoosterDag(i, datetime.date(2020,1, 1)))

In [287]:
# code voor het invullen van crew bij een tascrewLijn waarbij nog niet wordt gelet op of mensen al bezet zijn.

for j in range(0, len(dfSprint1)):
    inplannen = False
    i = 0
    gevondenDatum = False
    print("j: ",j)
    eersteDatum = datetime.date(2020,1, 1)
    
    while inplannen == False:
        if OpdrachtUitvoerenPersoonBool(dfSprint1.iloc[j,rangeSkills], dfCrewCombined.iloc[i,0:13]): # !TODO min en max mensen
            print("task met crew gevonden")
            dfSprint1.at[j,'Crew'] = np.array(dfCrewCombined.iloc[i,14:20])
            print(dfSprint1.Crew[j])
            for crewLijn in dfCrewCombined.iloc[i,14:20].tolist():
                if crewLijn ==20:
                    break
                
                #if min(Lijstmogelijkheden(crewLijn, dfSprint1.Duration[j]), key=attrgetter('datum')).datum <= eersteDatum:
                if VindRoosterdag(crewLijn, eersteDatum).urenTeGaan >= dfSprint1.Duration[j]:
                    gevondenDatum = True
                else:
                    gevondenDatum = False
                    break
                
                print("datum gevonden ", gevondenDatum)
            if gevondenDatum:
                for crewLijn in dfCrewCombined.iloc[i,14:20].tolist():
                    if crewLijn ==20:
                        break
                    dag = VindRoosterdag(crewLijn, eersteDatum)
                    print("dag gevonden ", eersteDatum)
                    print("uren die nog moeten worden gemaakt: ", dag.urenTeGaan)
                    dag.urenTeGaan = dag.urenTeGaan - dfSprint1.Duration[j]
                    print(dag)
                    
                    print(dfSprint1.Task[j], dfSprint1.Duration[j])
                    if type(dag.projectUren) == list:
                        dag.projectUren.append(ProjectUren(dfSprint1.Task[j], dfSprint1.Duration[j]))
                    else:
                        dag.projectUren = add_project(dag.projectUren, [dfSprint1.Task[j], dfSprint1.Duration[j]])
#                     werkdagen.append(RoosterDag(k, eersteDatum, 7- dfSprint1.Duration[j] , [dfSprint1.Task[j], dfSprint1.Duration[j]]))
                i=0
                inplannen = True
            else:
                print("volgende datum vinden")
                eersteDatum = eersteDatum + datetime.timedelta(days=1) # misschien wordt dit te vaak gedaan.
                for crewLijn in range(0, len(dfCrew)):
                    werkdagen.append(RoosterDag(crewLijn, eersteDatum))
        else:
            i = i + 1


j:  0
task met crew gevonden
[ 9  0 20 20 20 20]
datum gevonden  True
datum gevonden  True
dag gevonden  2020-01-01
uren die nog moeten worden gemaakt:  7
S1C1T1A 2.0
dag gevonden  2020-01-01
uren die nog moeten worden gemaakt:  7
S1C1T1A 2.0
j:  1
task met crew gevonden
[11 10 20 20 20 20]
datum gevonden  True
datum gevonden  True
dag gevonden  2020-01-01
uren die nog moeten worden gemaakt:  7
S1C1T1B 2.0
dag gevonden  2020-01-01
uren die nog moeten worden gemaakt:  7
S1C1T1B 2.0
j:  2
task met crew gevonden
[10  5 20 20 20 20]
datum gevonden  True
datum gevonden  True
dag gevonden  2020-01-01
uren die nog moeten worden gemaakt:  5.0
S1C1T2 2.0
dag gevonden  2020-01-01
uren die nog moeten worden gemaakt:  7
S1C1T2 2.0
j:  3
task met crew gevonden
[10  5 20 20 20 20]
datum gevonden  True
datum gevonden  True
dag gevonden  2020-01-01
uren die nog moeten worden gemaakt:  3.0
S1C1T3 2.0
dag gevonden  2020-01-01
uren die nog moeten worden gemaakt:  5.0
S1C1T3 2.0
j:  4
task met crew gevond

Misschien moeten we aan het begin geen eens moeite doen met hoe we de pauzes indelen. gewoon iemand zeven uur per dag laten werken. en dan de volgende dag iets laten beginnen dat pas kan beginnen als de andere taak af is. dan kunnen we later dat nog optimaliseren.

https://www.daniweb.com/programming/software-development/code/216631/a-list-of-class-objects-python
Goede site om een beetje te kijken hoe je met de lijst van objecten moet werken.

In [ ]:
# werkdagen = []
# werkdagen.append(RoosterDag("Robbert", "3-10-1996", 5, ["S1", 2]))
# werkdagen.append(RoosterDag("Robber", "3-10-1996", 4, ["S1", 2]))
# werkdagen.append(RoosterDag("Robert", "3-10-1996", 3, ["S1", 2]))

# werkdagen.append(RoosterDag("Robbert", "2-10-1996", 3, ["S1",3]))

# werkdagen.append(RoosterDag("Robbert", "3-9-1996"))

In [ ]:
# werkdagen.sort(key=operator.attrgetter('datum'))

In [220]:
min(Lijstmogelijkheden(9,2), key=attrgetter('datum')).datum

datetime.date(2020, 1, 2)

In [246]:
werkdagen[10].projectUren

In [244]:
dfSprint1.Task[2]


'S1C1T2'

In [252]:
VindRoosterdag(5,datetime.date(2020,1, 1)).projectUren.project

'x'

In [271]:
werkdagen[10].urenTeGaan

-4.0